In [ ]:
import importlib

import models.c_model
import models.model

import models.s_model
import models.d_model
import models.d2_model
import models.n_model
import scripts.trainer 
import scripts.train
import scripts.eval
import scripts.test
import customs.focal_loss
import scripts.data_loader
importlib.reload(scripts.data_loader)
importlib.reload(models.n_model)
importlib.reload(models.d2_model)
importlib.reload(customs.focal_loss)
importlib.reload(scripts.test)
importlib.reload(scripts.eval)
importlib.reload(models.d_model)
importlib.reload(scripts.train)
importlib.reload(scripts.trainer)
importlib.reload(models.c_model) 
importlib.reload(models.model) 
importlib.reload(models.try2) 
importlib.reload(models.s_model)

from models.d_model import MergeNet
from models.s_model import S_model
from models.c_model import Network
from models.model import MixerMLP, initialize_weights
from models.try2 import MLPMixer
from models.d2_model import MergeNet2
from models.n_model import DenseNet

from customs.focal_loss import FocalLoss

import wandb
import torch
import gc

from scripts.train import train
from scripts.eval import eval
from scripts.test import test
from scripts.trainer import Trainer
from scripts.data_loader import AudioDatasetModule

from torchsummaryX import summary
import matplotlib.pyplot as plt

In [ ]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

config = {
    'subset': 1.0, # Subset of dataset to use (1.0 == 100% of data)
    'context': 24,  # 30
    'activations': 'Swish',
    'learning_rate': 1e-3,
    'dropout': 0.3,
    'optimizers': 'AdamW',
    'scheduler': 'OneCycleLR',
    'epochs': 100,       # 30
    'batch_size': 2048, # 1024, 500
    'patience': 30,  
    'save_every': 1,
    'weight_decay': 0.01,
    'weight_initialization': 'xavier_normal', # e.g kaiming_normal, kaiming_uniform, uniform, xavier_normal or xavier_uniform
    'augmentations': 'FreqMask', # Options: ["FreqMask", "TimeMask", "Both", null]
    'freq_mask_param': 4, #4
    'time_mask_param': 8
 }

device = "cuda"

def clean_cache(device):
    if device == "mps":
        torch.mps.empty_cache()
    elif device == "cuda":
        torch.cuda.empty_cache()

In [ ]:
dm = AudioDatasetModule(
    root="../autodl-fs/data",
    phonemes=PHONEMES,
    train_partition="train-clean-100",
    val_partition="dev-clean",
    test_partition="test-clean",
    batch_size=config["batch_size"],
    config=config,
    num_workers=14,
    pin_memory=True
)

dm.initialize(mode="fit")
train_loader = dm.train_dataloader()
val_loader   = dm.val_dataloader()

dm.initialize(mode="test")
test_loader = dm.test_dataloader()

print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*28)
print("Output symbols : ", len(PHONEMES))

print("batches = {}".format(len(train_loader)))
print("batches = {}".format(len(val_loader)))
print("batches = {}".format(len(test_loader)))

In [ ]:
model_n = DenseNet(arch=(4096, 2048, 1024, 1024, 750, 512), 
                   num_ouputs=round(len(PHONEMES)), 
                   dropout=(0.2, 0.15, 0.15, 0.15, 0.05, 0)).to(device)

model = model_n
model_name = "model"

In [ ]:
torch.cuda.empty_cache()
inputs, _ = next(iter(train_loader))
model.apply_init(inputs.to(device), initialize_weights)

print(inputs.shape)

criterion = torch.nn.CrossEntropyLoss()
# criterion = FocalLoss(gamma=1.5)

optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=config["weight_decay"])

# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=len(train_loader), T_mult=1, 
#                                                                  eta_min = 0.0001)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=round(len(train_loader) * 1.2), 
#                                                                  eta_min = 0.00005)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3 * len(train_loader), gamma=0.9)

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=2e-3, 
    total_steps = 20 * len(train_loader), 
    pct_start = 0.15, 
    anneal_strategy="cos"
)

scaler = torch.amp.GradScaler('cuda', enabled=True) 

clean_cache(device)
gc.collect()

start=0
best_val_acc = 0

In [ ]:
# wandb login 
wandb.login(key="......")

In [ ]:
# For test
wandb.unwatch(model)
test_Trainer = Trainer(config["epochs"], criterion, optimizer, scheduler,
                       config["patience"], config["save_every"], model_name, device=device, scaler=scaler)
test_Trainer.fit(model, train_loader, val_loader, log=False, save_best=False,
                 checkpoints=False)

In [ ]:
# Create wandb run
run = wandb.init(
    name    = f"{model_name}_run_1", ### set run names
    reinit  = True, ### Allows reinitalizing runs when re-running this cell
    #id     = "", ### Insert specific run id here if resuming a previous run
    #resume = "must", ### need this to resume previous runs, but comment out reinit = True when using this
    project = "HW1P2", ### Project name
    group=f"{model_name}", 
    config=config
)

In [ ]:
clean_cache(device)
gc.collect()
wandb.watch(model, log="all")

trainer = Trainer(config["epochs"], criterion, optimizer, scheduler, 
                       config["patience"], config["save_every"], model_name, 
                       start=start, best_val_acc=best_val_acc, log_freq=20, 
                       device=device, scaler=scaler)
trainer.fit(model, train_loader, val_loader, save_best=True, checkpoints=True)

In [ ]:
"----------------------------------------------------------------------"
"----------------------------Resume Run--------------------------------"
"----------------------------------------------------------------------"

In [ ]:
# Load checkpoint
checkpoint = torch.load("checkpoints/model/model_epoch_40.pth")

model.load_state_dict(checkpoint["model_state_dict"])
# optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
# scheduler.load_state_dict(checkpoint["scheduler_state_dict"])

start=40
best_val_acc = 0.86

In [ ]:
wandb.init(
    name    = f"{model_name}_run_1",
    reinit  = True,
    id      = '......',   ### ID for the run
    resume  = "must", 
    project = "HW1P2",
    group=f"{model_name}",
    config=config
)